In [1]:
"""
This is an apsw virtual table interface to Cinema Spec C.

In here are defined 3 things:

1. The cursor
2. The table
3. The module

You have to tell apsw about your module, which will
then allow you to attach to a cinema database and
be able to create cursors on it. The minimal setup is:

    # given a table name and path to the database with the info.json
    import apsw
    import cinemasqlpy.c as C

    conn = apsw.Connection(':memory:')
    cursor = conn.cursor()
    module = A.Module()
    conn.createmodule('CinemaC', module)

    cursor.execute('drop table if exists %s' % table)
    cursor.execute('create virtual table %s using CinemaC(%s)' % (table, path))
"""

import json
from os import path

class Cursor:
    """An apsw cursor implementation to a Cinema Spec C database."""

    # table data
    _cols = None
    _pattern = None
    _path = None
    
    # cursor data
    _rowid = None
    _limit = None
    _offsets = None
   
    def __init__(self, cols, pattern, path):
        """Takes the columns, filename pattern, and path to the cinema database."""

        self._cols = cols
        self._pattern = pattern
        self._path = path
        
        offsets = [1] * len(cols)
        for i in range(0, len(cols) - 1):
            offsets[i+1] = offsets[i] * len(cols[i])
        self._offsets = offsets
        self._limit = self._offsets[-1] * len(cols[-1])
   
    def Close(self):
        pass
   
    # Column index is used to figure out to index into the columns,
    # based on row id.
    def _column_index(self, i):
        return ((self._rowid - 1) // self._offsets[i]) % len(self._cols[i])
    
    def Column(self, num):
        if num == 0:
            args = [self._cols[i][self._column_index(i)] for i in range(0, len(self._cols))]
            return self._pattern.format(*args)
        elif num == 1:
            args = [self._cols[i][self._column_index(i)] for i in range(0, len(self._cols))]
            return self._path + self._pattern.format(*args)           
        else:
            num = num - 2
            return self._cols[num][self._column_index(num)]
    
    def Eof(self):
        return self._rowid > self._limit
    
    def Filter(self, num, name, cons):
        self._rowid = 1
    
    def Next(self):
        self._rowid = self._rowid + 1
    
    def Rowid(self):
        return rowid
    
class Table:
    """An apsw table implementation to a Cinema Spec C database."""

    # table data
    _cols = None
    _pattern = None
    _path = None
    
    def __init__(self, cols, pattern, path):
        """Takes the columns, filename pattern, and path to the cinema database."""

        self._cols = cols
        self._pattern = pattern
        self._path = path
    
    def BestIndex(self, cons, order):
        # *TODO* create some indices
        #        because it's going to be slow (everything will be a scan)
        return None
    
    def Destroy(self):
        pass
    
    def Disconnect(self):
        pass
    
    # This is where the cursor is actually created for apsw.
    def Open(self):
        return Cursor(self._cols, self._pattern, self._path)
   
    # *Unimplemented apsw functions*
    # def Begin(self)
    # def Commit(self)
    # def FindFunction(self, name, args)
    # def Rename(self)
    # def Rollback(self)
    # def Sync(self)
    # def UpdateChangeRow(self, rowid, newid, vals)
    # def UpdateDeleteRow(self, rowid)
    # def UpdateInsertRow(self, rowid, vals)
    
def read_cinema(fn):
    """Parse a Cinema Spec C database schema.
       
       Parameters
       ----------
           fn : string
           Path to the database that contains the info.json

       Side-effects
       ------------
           None.

       Returns
       -------
           (valstr, cols, pattern)
           valstr : string
               A SQL substring for describing the Cinema DB as values schema
           cols : a list of lists of values 
               The values of the range parameters in the Cinema DB
           pattern : string
               The filename pattern for columns to image filename
    """

    t = json.load(open(fn + '/info.json'))

    # *TODO* validate metadata header
    
    # scan for the parameters and only get the 'range' ones
    name_pairs = []
    for k, v in t['arguments'].items():
        if v['type'] == 'range':
            name_pairs.append((k, v['label']))
        # *TODO* throw an exception if not 'range'?
    name_pairs = sorted(name_pairs, key=lambda x: x[0])

    # create a pattern based on column order
    kwargs = {}
    for i, pair in zip(range(0, len(name_pairs)), name_pairs):
        kwargs[pair[0]] = '{' + str(i) + '}'
    pattern = t['name_pattern'].format(**kwargs)
    
    # create the (value) string for the create table
    # put the values into columns ordered by name_pairs (same order as value string)
    valstr = ' (filename text,full_path text,'
    cols = []
    for pair in name_pairs:
        valstr = valstr + pair[1] + ' real,'
        cols.append(sorted(t['arguments'][pair[0]]['values']))
    valstr = valstr[:-1] + ')'

    return (valstr, cols, pattern)

class Module:
    """An apsw virtual table implementation to a Cinema Spec C database."""

    _valstr = None
    _cols = None
    _pattern = None
    _path = None
    
    def __init__(self):
        pass
    
    def Create(self, conn, mod, db, name, *vtargs):
        """Takes an apsw connection, module, database, tablename, and additional
           creation arguments which are all passed via the SQL create virtual table
           command. Returns a tuple of (SQL create table string, Cinema C apsw table object).
        """

        self._path = path.dirname(vtargs[0]) 
        if self._path[-1] != '/':
            self._path = self._path + '/'
        self._valstr, self._cols, self._pattern = read_cinema(self._path)
        
        return ('create table ' + name + self._valstr, Table(self._cols, self._pattern, self._path))
    
    Connect = Create
    


In [2]:
"""
This is an apsw virtual table interface to Cinema Spec C.

In here are defined 3 things:

1. The cursor
2. The table
3. The module

You have to tell apsw about your module, which will
then allow you to attach to a cinema database and
be able to create cursors on it. The minimal setup is:

    # given a table name and path to the database with the info.json
    import apsw
    import cinemasqlpy.c as C

    conn = apsw.Connection(':memory:')
    cursor = conn.cursor()
    module = A.Module()
    conn.createmodule('CinemaC', module)

    cursor.execute('drop table if exists %s' % table)
    cursor.execute('create virtual table %s using CinemaC(%s)' % (table, path))
"""

import json
from os import path

class CursorTables:
    """An apsw cursor implementation to a Cinema Spec C database."""

    # table data
    _tables = None
    
    # cursor data
    _rowid = None
    _limit = None
   
    def __init__(self, tables):
        """Takes the tables (list of table names) in the cinema database."""

        self._tables = tables        
        self._limit = len(tables)
   
    def Close(self):
        pass
    
    def Column(self, num):
        return self._tables[self._rowid - 1]
    
    def Eof(self):
        return self._rowid > self._limit
    
    def Filter(self, num, name, cons):
        self._rowid = 1
    
    def Next(self):
        self._rowid = self._rowid + 1
    
    def Rowid(self):
        return rowid
    
class TableTables:
    """An apsw table implementation to a Cinema Spec C database of the list of tables."""

    # table data
    _tables = None
    
    def __init__(self, tables):
        """Takes the tables (list of table names) in the cinema database."""

        self._tables = tables
    
    def BestIndex(self, cons, order):
        # *TODO* create some indices
        #        because it's going to be slow (everything will be a scan)
        return None
    
    def Destroy(self):
        pass
    
    def Disconnect(self):
        pass
    
    # This is where the cursor is actually created for apsw.
    def Open(self):
        return CursorTables(self._tables)
   
    # *Unimplemented apsw functions*
    # def Begin(self)
    # def Commit(self)
    # def FindFunction(self, name, args)
    # def Rename(self)
    # def Rollback(self)
    # def Sync(self)
    # def UpdateChangeRow(self, rowid, newid, vals)
    # def UpdateDeleteRow(self, rowid)
    # def UpdateInsertRow(self, rowid, vals)

def read_cinema_tables(fn):
    """Parse a Cinema Spec C database list of tables.
       
       Parameters
       ----------
           fn : string
           Path to the database that contains the info.json

       Side-effects
       ------------
           None.

       Returns
       -------
           (valstr, tables)
           valstr : string
               A SQL substring for describing schema of table names
           tables : [string]
               A list of table names
    """

    t = json.load(open(fn + '/info.json'))

    # *TODO* validate metadata header
    
    # scan for the parameters and only get the 'range' ones
    name_pairs = []
    for k, v in t['parameter_list'].items():
        if v['type'] == 'range' or v['type'] == 'hidden' or v['type'] == :
            name_pairs.append((k, v['label']))
        # *TODO* throw an exception if not 'range'?
    name_pairs = sorted(name_pairs, key=lambda x: x[0])

    # create a pattern based on column order
    kwargs = {}
    for i, pair in zip(range(0, len(name_pairs)), name_pairs):
        kwargs[pair[0]] = '{' + str(i) + '}'
    pattern = t['name_pattern'].format(**kwargs)
    
    # create the (value) string for the create table
    # put the values into columns ordered by name_pairs (same order as value string)
    valstr = ' (filename text,full_path text,'
    cols = []
    for pair in name_pairs:
        valstr = valstr + pair[1] + ' real,'
        cols.append(sorted(t['arguments'][pair[0]]['values']))
    valstr = valstr[:-1] + ')'

    return (valstr, cols, pattern)

class Module:
    """An apsw virtual table implementation to a Cinema Spec C database."""

    _valstr = None
    _tables = None
    
    def __init__(self):
        pass
    
    def Create(self, conn, mod, db, name, *vtargs):
        """Takes an apsw connection, module, database, tablename, and additional
           creation arguments which are all passed via the SQL create virtual table
           command. Returns a tuple of (SQL create table string, Cinema C apsw table object).
        """

        self._path = path.dirname(vtargs[0]) 
        if self._path[-1] != '/':
            self._path = self._path + '/'
        self._valstr, self._tables = read_cinema_tables(self._path)
        
        return ('create table ' + name + self._valstr, TableTables(self._tables))
    
    Connect = Create
    


In [3]:
json.load(open('/home/woodring/sqlcinema/test/rainbowsphere_C.cdb/image/info.json'))

{'constraints': {'Sphere1': {'vis': ['Sphere1']},
  'colorSphere1': {'vis': ['Sphere1']}},
 'metadata': {'camera_angle': [30.0],
  'camera_at': [[0.0, 0.0, 0.0]],
  'camera_eye': [[0.0, 0.0, 3.3338563951358404]],
  'camera_model': 'phi-theta',
  'camera_nearfar': [[2.305517831184482, 4.6363642410628785]],
  'camera_up': [[0.0, 1.0, 0.0]],
  'pipeline': [{'children': [],
    'id': '4850',
    'name': 'Sphere1',
    'parents': ['0'],
    'visibility': 1}],
  'store_type': 'FS',
  'type': 'composite-image-stack',
  'value_mode': 1,
  'version': '0.1'},
 'name_pattern': '{phi}/{theta}.png',
 'parameter_list': {'colorSphere1': {'default': 'Normals_0',
   'label': 'colorSphere1',
   'role': 'field',
   'type': 'hidden',
   'types': ['depth', 'luminance', 'value', 'value', 'value'],
   'valueRanges': {'Normals_0': [-0.9945219159126282, 0.9945219159126282],
    'Normals_1': [-0.9945219159126282, 0.9945219159126282],
    'Normals_2': [-1.0, 1.0]},
   'values': ['depth', 'luminance', 'Normals_0'